In [58]:
# 초기에 selenium은 install 해야할 수 있음
# !pip install selenium

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import datetime 
import sys

In [2]:
print(f'Python Version : {sys.version}')
print(f'Selenium webdriver Version : {webdriver.__version__}')
print(f'Pandas Version : {pd.__version__}')

Python Version : 3.8.3 (default, Jul  2 2020, 11:26:31) 
[Clang 10.0.0 ]
Selenium webdriver Version : 4.5.0
Pandas Version : 1.4.3


In [ ]:
# ‘chromedriver’은(는) Apple에서 악성 소프트웨어가 있는지 확인할 수 없기 때문에 열 수 없습니다. 가 뜨면
# !xattr -d com.apple.quarantine chromedriver

In [3]:
# 셀레니움 드라이버 시작
def driver_start():
    # user_agent
    user_agent = 'user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko'
    user_agent = 'Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36'

    # options setting
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument(user_agent) # Fake User Agent 설정
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
    chrome_options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
    chrome_options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
    chrome_options.add_argument('window-size={1000},{813}') #브라우저 사이즈 설정


    # driver start
    driver = webdriver.Chrome(executable_path='./chromedriver', options=chrome_options)
    return driver

In [4]:
# 스크롤 다운
def scroll():
    #스크롤 내리기 이동 전 위치
    scroll_location = driver.execute_script("return document.body.scrollHeight")

    while True:
        #현재 스크롤의 가장 아래로 내림
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

        #전체 스크롤이 늘어날 때까지 대기
        time.sleep(2)

        #늘어난 스크롤 높이
        scroll_height = driver.execute_script("return document.body.scrollHeight")

        #늘어난 스크롤 위치와 이동 전 위치 같으면(더 이상 스크롤이 늘어나지 않으면) 종료
        if scroll_location == scroll_height:
            break

        #같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
        else:
            #스크롤 위치값을 수정
            scroll_location = driver.execute_script("return document.body.scrollHeight")

In [26]:
def scroll_down(driver):
    """
    스크롤 끝까지 내리는 코드
    """
    SCROLL_PAUSE_SEC = 3
    # 스크롤 높이 가져옴
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # 1초 대기
        time.sleep(SCROLL_PAUSE_SEC)
        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    print("스크롤 다운 완료!")

In [37]:
# 채용공고 url 획득

def get_urls(driver, url):

    driver.get(url)
    time.sleep(3)
    
    # urls 저장
    position_urls = []
    
#     # 50인 이하 클릭
#     driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div/div/div[1]/div[1]/section/div/div/div/div/div[6]/div/button').click()
    
    scroll_down(driver)
    
    # 채용공고를 아래로 스크롤하여 더 추가할 경우 아래 주석을 풀고 실행, 오래걸림 주의
    position_length = len(driver.find_elements(By.CLASS_NAME, 'Card_className__u5rsb'))
            
    time.sleep(1)
    for i in range(position_length):
        position_url = driver.find_element(By.CLASS_NAME, 'List_List_container__JnQMS').find_element(By.CSS_SELECTOR, 'ul').find_elements(By.CSS_SELECTOR, 'li')[i].find_element(By.CSS_SELECTOR, 'div').find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
        position_urls.append(position_url)

    print("링크 개수 : ", end='')
    print(position_length)
        
    return position_urls

In [55]:
# 각 채용공고 별 상세 jd 획득
def crawling_wanted(position_urls, driver):
    results =[]
    for i in range(len(position_urls)):
#         print(f'{i+1} / {len(position_urls)}')
        driver.get(position_urls[i])  
        result = {
            '기업명': '',
            '공고 제목' : '',
            '산업분류' : '',
            '태그' : '',
            '마감일' : '',
            '주요업무' : '',
            '자격요건' : '',
            '우대사항' : '',
            '혜택 및 복지' : '',
            'URL' : '',
         }
        time.sleep(3)
        # 기업명, 공고 제목, URL
        
        try:
            company_name =  driver.find_elements(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div[1]/div[1]/section[2]/div[1]/h6/a')[0].text
            result['기업명'] = company_name
            job_title = driver.find_elements(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div[1]/div[1]/section[2]/h2')[0].text
            result['공고 제목'] = job_title
            industry = driver.find_elements(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div[1]/div[1]/section[3]/button[1]/div[2]/h6')[0].text
            result['산업분류'] = industry
            tag = driver.find_elements(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div[1]/div[1]/section[2]/div[3]')[0].text
            result['태그'] = tag
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        
            # 마감일 동적 페이지 로딩을 위해 페이지 이동
            action = ActionChains(driver)
            element = driver.find_elements(By.CLASS_NAME, 'WarningHeader_WarningHeader__F1ikW')[0]
            action.move_to_element(element).perform()
            time.sleep(0.5)
        
        
#             마감일
            date = driver.find_elements(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div[1]/div[1]/div[2]/section[2]/div[1]/span[2]')[0].text
            result['마감일'] = date
            result['URL'] = position_urls[i]

            # JD - title
            driver.find_elements(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div[1]/div[1]/div[2]/section[1]/h6')
            size = len(driver.find_elements(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div[1]/div[1]/div[2]/section[1]/h6'))
            
            for j in range(1, size+1):
                title = driver.find_elements(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div[1]/div[1]/div[2]/section[1]/h6')[j-1].text
                content = driver.find_elements(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div[1]/div[1]/div[2]/section[1]/p')[j].text
                result[title] = content
                results.append(result)
            
        except:
            print("채용 정보를 가져오지 못 했습니다.")
            continue
        
    return results

In [7]:
def make_df(results):
    wanted_crawling = pd.DataFrame(results)
    # 중복체크
    if wanted_crawling.duplicated('URL').sum():
        print(f'Drop Duplicated : {wanted_crawling.duplicated("URL").sum()}')
        wanted_crawling = wanted_crawling.drop_duplicates()
    return wanted_crawling

In [8]:
def save_df(wanted_crawling):
    # Data Frame save
    today = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    wanted_crawling.to_csv(f'./datas/{today}_wanted_crawling.csv', index=False, line_terminator = '\n')
    print('Save 완료!')

In [18]:
# # colab only
# from google.colab import auth
# import gspread
# from google.auth import default

# # def make_url_with_spreadsheet():
# auth.authenticate_user()
# creds, _ = default()
# gc = gspread.authorize(creds)

# gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1AO2L-s48ny3A2UuY21I38ax8zQ8CIMJRP33_yyD7ZCw')
# sheet_names = ['job']
# sheets = gsheets.worksheet(sheet_names[0]).get_all_values()
# df = pd.DataFrame(sheets)
# df[:][2]

# print('링크 로딩 완료!')

ModuleNotFoundError: No module named 'google.colab'

In [38]:
driver = driver_start()

# https://docs.google.com/spreadsheets/d/1AO2L-s48ny3A2UuY21I38ax8zQ8CIMJRP33_yyD7ZCw/edit#gid=0

MLE_url = ['https://www.wanted.co.kr/search?query=ML',
 'https://www.wanted.co.kr/search?query=machine%20learning',
 'https://www.wanted.co.kr/search?query=%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D',
 'https://www.wanted.co.kr/search?query=deep%20learning',
 'https://www.wanted.co.kr/search?query=%EB%94%A5%EB%9F%AC%EB%8B%9D',
 'https://www.wanted.co.kr/search?query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5']
DS_url = ['https://www.wanted.co.kr/search?query=%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%82%AC%EC%9D%B4%EC%96%B8%ED%8B%B0%EC%8A%A4%ED%8A%B8',
 'https://www.wanted.co.kr/search?query=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%82%AC%EC%9D%B4%EC%96%B8%ED%8B%B0%EC%8A%A4%ED%8A%B8',
 'https://www.wanted.co.kr/search?query=data%20scientist',
 'https://www.wanted.co.kr/search?query=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EA%B3%BC%ED%95%99%EC%9E%90']
DA_url = ['https://www.wanted.co.kr/search?query=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D%EA%B0%80',
 'https://www.wanted.co.kr/search?query=data%20analyst',  
 'https://www.wanted.co.kr/search?query=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%95%A0%EB%84%90%EB%A6%AC%EC%8A%A4%ED%8A%B8']
DE_url = ['https://www.wanted.co.kr/search?query=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%97%94%EC%A7%80%EB%8B%88%EC%96%B4',
 'https://www.wanted.co.kr/search?query=data%20engineer',
 'https://www.wanted.co.kr/search?query=%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%97%94%EC%A7%80%EB%8B%88%EC%96%B4']

for i in range(len(MLE_url)):
    if(i == 0): MLE_position_urls = get_urls(driver, MLE_url[i])
    else: MLE_position_urls.extend(get_urls(driver, MLE_url[i]))
        
for i in range(len(DS_url)):
    if(i == 0): DS_position_urls = get_urls(driver, DS_url[i])
    else: DS_position_urls.extend(get_urls(driver, DS_url[i]))
        
for i in range(len(DA_url)):
    if(i == 0): DA_position_urls = get_urls(driver, DA_url[i])
    else: DA_position_urls.extend(get_urls(driver, DA_url[i]))
        
for i in range(len(DE_url)):
    if(i == 0): DE_position_urls = get_urls(driver, DE_url[i])
    else: DE_position_urls.extend(get_urls(driver, DE_url[i]))


<ipython-input-3-46708426ca63>:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver', options=chrome_options)


스크롤 다운 완료!
링크 개수 : 66
스크롤 다운 완료!
링크 개수 : 31
스크롤 다운 완료!
링크 개수 : 50
스크롤 다운 완료!
링크 개수 : 18
스크롤 다운 완료!
링크 개수 : 45
스크롤 다운 완료!
링크 개수 : 39
스크롤 다운 완료!
링크 개수 : 3
스크롤 다운 완료!
링크 개수 : 30
스크롤 다운 완료!
링크 개수 : 41
스크롤 다운 완료!
링크 개수 : 0
스크롤 다운 완료!
링크 개수 : 60
스크롤 다운 완료!
링크 개수 : 59
스크롤 다운 완료!
링크 개수 : 3
스크롤 다운 완료!
링크 개수 : 105
스크롤 다운 완료!
링크 개수 : 94
스크롤 다운 완료!
링크 개수 : 8


In [43]:
print(len(MLE_position_urls))
print(len(DS_position_urls))
print(len(DA_position_urls))
print(len(DE_position_urls))

249
74
122
207


In [46]:
MLE_position_urls

['https://www.wanted.co.kr/wd/63547',
 'https://www.wanted.co.kr/wd/94316',
 'https://www.wanted.co.kr/wd/117605',
 'https://www.wanted.co.kr/wd/90288',
 'https://www.wanted.co.kr/wd/61066',
 'https://www.wanted.co.kr/wd/123380',
 'https://www.wanted.co.kr/wd/114176',
 'https://www.wanted.co.kr/wd/106875',
 'https://www.wanted.co.kr/wd/36413',
 'https://www.wanted.co.kr/wd/106594',
 'https://www.wanted.co.kr/wd/97166',
 'https://www.wanted.co.kr/wd/92460',
 'https://www.wanted.co.kr/wd/126742',
 'https://www.wanted.co.kr/wd/132230',
 'https://www.wanted.co.kr/wd/111272',
 'https://www.wanted.co.kr/wd/110126',
 'https://www.wanted.co.kr/wd/107285',
 'https://www.wanted.co.kr/wd/64640',
 'https://www.wanted.co.kr/wd/123360',
 'https://www.wanted.co.kr/wd/93656',
 'https://www.wanted.co.kr/wd/96372',
 'https://www.wanted.co.kr/wd/96374',
 'https://www.wanted.co.kr/wd/100508',
 'https://www.wanted.co.kr/wd/100509',
 'https://www.wanted.co.kr/wd/100512',
 'https://www.wanted.co.kr/wd/83267'

In [56]:
# if chrome window was dead, you need to do "driver = driver_start()" agian"
driver = driver_start()

# AIB_result = list()
MLE_result = list()
DS_result = list()
DA_result = list()
DE_result = list()

# MLE_position_urls
# DS_position_urls
# DA_position_urls
# DE_position_urls
        
MLE_result = crawling_wanted(MLE_position_urls, driver)
wanted_crawling_MLE = make_df(MLE_result)
save_df(wanted_crawling_MLE)

<ipython-input-3-46708426ca63>:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver', options=chrome_options)


Drop Duplicated : 962
Save 완료!


In [57]:
DS_result = crawling_wanted(DS_position_urls, driver)
wanted_crawling_DS = make_df(DS_result)
save_df(wanted_crawling_DS)

DA_result = crawling_wanted(DA_position_urls, driver)
wanted_crawling_DA = make_df(DA_result)
save_df(wanted_crawling_DA)

DE_result = crawling_wanted(DE_position_urls, driver)
wanted_crawling_DE = make_df(DE_result)
save_df(wanted_crawling_DE)

Drop Duplicated : 293
Save 완료!
Drop Duplicated : 466
Save 완료!
Drop Duplicated : 809
Save 완료!
